# imdb dataset

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

In [ ]:
vocabulary_size = 15000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary_size)

In [ ]:
print(x_train[0])

In [ ]:
word2num = imdb.get_word_index()

In [ ]:
num2word = {v:k for k,v in word2num.items()}

In [ ]:
print(" - ".join(num2word[x] for x in x_train[0]))

In [ ]:
y_train[0]

In [ ]:
x_train.shape, x_train.dtype

### padding the data

In [ ]:
maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [ ]:
x_train[2]

In [ ]:
maxlen = 60
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

### LSTM using Keras

In [ ]:
model = Sequential()
model.add(Embedding(vocabulary_size, 128))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from IPython.display import SVG, display
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=2,
          validation_data=(x_test, y_test))

In [ ]:
score, acc = model.evaluate(x_test, y_test, batch_size=32)
print(score, acc)

### Tensorflow implementation
https://www.tensorflow.org/tutorials/recurrent

In [ ]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence

vocabulary_size = 15000
maxlen = 60
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocabulary_size)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.float32(y_train)[:, None]
y_test = np.float32(y_test)[:, None]

In [ ]:
# tf dataset
import tensorflow as tf

batch_size = 40
lstm_size = 64

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=10000).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types,
                                           train_data.output_shapes)
train_data_init = iterator.make_initializer(train_data)
test_data_init = iterator.make_initializer(test_data)
X, Y_ = iterator.get_next()

In [ ]:
# word embedding
embedding_matrix = tf.Variable(tf.random_uniform([vocabulary_size, 128], -1.0, 1.0))
word_embeddings = tf.nn.embedding_lookup(embedding_matrix, X)

# LSTM cell
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

# dropout wrapper
keep_prob = tf.placeholder("float", name="keep_prob")
lstm = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob, state_keep_prob=keep_prob)

In [ ]:
# LSTM network
lstm_input =[word_embeddings[:, i] for i in range(maxlen)]
init_state = state = lstm.zero_state(batch_size, dtype=tf.float32)
for x in lstm_input:
    lstm_output, state = lstm(x, state)
# or use tf.split, tf.contrib.rnn.static_rnn

In [ ]:
lstm_output

In [ ]:
# readout
Y = tf.layers.dense(lstm_output, 1, activation=tf.nn.sigmoid)

# loss function and training
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_, logits=Y))
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
prediction = tf.floor(Y+0.5)
accuracy = tf.reduce_mean(tf.cast(tf.less(tf.abs(Y-Y_), 0.5), tf.float32))


In [ ]:
import time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
t0 = time.time()
for epoch in range(10):
    sess.run(train_data_init)
    while True:
        try:
            sess.run(train_step, {keep_prob: 0.8 })
        except tf.errors.OutOfRangeError:
            print("End of epoch", epoch, "time:", time.time()-t0)
            break
    sess.run(test_data_init)
    test_accuracy = np.mean([sess.run(accuracy,{keep_prob: 1.0 }) for i in range(100)])
    print("Epoch %d, validation accuracy %g"%(epoch, test_accuracy))
sess.close()